<figure>
  <img src="https://github.com/v-iashin/video_features/raw/master/docs/_assets/i3d.png" width="300" />
</figure>

The `video_features` library allows you to extract features from 
raw videos in parallel with multiple GPUs. 
It supports several extractors that capture visual appearance, 
optical flow, and audio features. See more details in the 
[GitHub repository](https://github.com/v-iashin/video_features).

See more feature extraction examples in colaboratory notebooks:
* [![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/1Zd7r8uKGLGSxlil4PPnXk_4I3KOsjPpO?usp=sharing) – CLIP
* [![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/1HUlYcOJf_dArOcAaR9jaQHuM5CAZiNZc?usp=sharing) – S3D
* [![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/1LKoytZmNxtC-EuCp7pHDM6sFvK1XdwlW?usp=sharing) – I3D
* [![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/1csJgkVQ3E2qOyVlcOM-ACHGgPBBKwE2Y?usp=sharing) – R(2+1)D 
* [![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/18I95Rn1B3a2ISfD9b-o4o93m3XuHbcIY?usp=sharing) – RAFT
* [![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/17VLdf4abQT2eoMjc6ziJ9UaRaOklTlP0?usp=sharing) – ResNet
* [![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/1r_8OnmwXKwmH0n4RxBfuICVBgpbJt_Fs?usp=sharing) – VGGish

In [ ]:
! git clone https://github.com/seejiafong/video_features.git
! pip install omegaconf==2.0.6

Cloning into 'video_features'...
remote: Enumerating objects: 1213, done.
remote: Counting objects: 100% (343/343), done.
remote: Compressing objects: 100% (147/147), done.


In [ ]:
import os
from google.colab import drive
drive.mount('/content/gdrive')
#os.chdir('/content/gdrive/My Drive/iss/itsspm/i3d/')

In [ ]:
%cd video_features

In [ ]:
from models.i3d.extract_i3d import ExtractI3D
from utils.utils import build_cfg_path
from omegaconf import OmegaConf
import torch

device = 'cuda' if torch.cuda.is_available() else 'cpu'
torch.cuda.get_device_name(0)

In [ ]:
'''
import glob
path = "/content/gdrive/My Drive/iss/itsspm/FineDiving_videos/_entry/*.mp4"
for file in glob.glob(path):
   print(file)
'''

In [ ]:
import numpy as np
import glob
# Select the feature type
feature_type = 'i3d'

# Load and patch the config
args = OmegaConf.load(build_cfg_path(feature_type))
args.video_paths = ['/content/gdrive/My Drive/iss/itsspm/i3d/inputvid/v_GGSY1Qvo990.mp4']
# args.show_pred = True
args.stack_size = 12
args.step_size = 4 
# args.extraction_fps = 25
args.flow_type = 'raft' # 'pwc' is not supported on Google Colab (cupy version mismatch)
args.streams = 'rgb'

# Load the model
extractor = ExtractI3D(args)

# Extract features
folder_paths = ["/content/gdrive/My Drive/iss/itsspm/i3d/96Frames/test/*.mp4", 
                "/content/gdrive/My Drive/iss/itsspm/i3d/96Frames/train/*.mp4"]
for path in folder_paths:
  count = 1
  for video_path in glob.glob(path):
      vidname = video_path.replace('.mp4','')
      #if (not os.path.exists(vidname+'_rgb.npy')):
      print(f'{count} : Extracting for {video_path}')
      feature_dict = extractor.extract(video_path)
      numWindows = (96-args.stack_size)/args.step_size
      for k, v in feature_dict.items():
        if (k=='rgb'):
          outfile = vidname+'_'+k+'.npy'
          v=v.reshape(1,int(numWindows*1024))
          np.save(outfile, v)
          count = count + 1



1 : Extracting for /content/gdrive/My Drive/iss/itsspm/i3d/96Frames/test/04_9_all.mp4
2 : Extracting for /content/gdrive/My Drive/iss/itsspm/i3d/96Frames/test/FINADivingWorldCup2021_Men3m_final_r1_0_all.mp4
3 : Extracting for /content/gdrive/My Drive/iss/itsspm/i3d/96Frames/test/05_61_all.mp4
4 : Extracting for /content/gdrive/My Drive/iss/itsspm/i3d/96Frames/test/01_124_all.mp4
5 : Extracting for /content/gdrive/My Drive/iss/itsspm/i3d/96Frames/test/09_69_all.mp4
6 : Extracting for /content/gdrive/My Drive/iss/itsspm/i3d/96Frames/test/22_14_all.mp4
7 : Extracting for /content/gdrive/My Drive/iss/itsspm/i3d/96Frames/test/14_40_all.mp4
8 : Extracting for /content/gdrive/My Drive/iss/itsspm/i3d/96Frames/test/10_41_all.mp4
9 : Extracting for /content/gdrive/My Drive/iss/itsspm/i3d/96Frames/test/07_17_all.mp4
10 : Extracting for /content/gdrive/My Drive/iss/itsspm/i3d/96Frames/test/01_140_all.mp4
11 : Extracting for /content/gdrive/My Drive/iss/itsspm/i3d/96Frames/test/07_18_all.mp4
12 : E

In [ ]:
#! pip freeze